# When Papa Atom Met Mama Atom
> A test of Bokeh and Altair in Fastpages.

- toc: true 
- badges: true
- comments: true
- categories: [elements, altair]
- image: images/chart-preview.png

# Periodic Table in Bokeh

There shall be elements

In [1]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.sampledata.periodic_table import elements
from bokeh.transform import dodge, factor_cmap

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
periods = ["I", "II", "III", "IV", "V", "VI", "VII"]
groups = [str(x) for x in range(1, 19)]

df = elements.copy()
df["atomic mass"] = df["atomic mass"].astype(str)
df["group"] = df["group"].astype(str)
df["period"] = [periods[x-1] for x in df.period]
df = df[df.group != "-"]
df = df[df.symbol != "Lr"]
df = df[df.symbol != "Lu"]

cmap = {
    "alkali metal"         : "#a6cee3",
    "alkaline earth metal" : "#1f78b4",
    "metal"                : "#d93b43",
    "halogen"              : "#999d9a",
    "metalloid"            : "#e08d49",
    "noble gas"            : "#eaeaea",
    "nonmetal"             : "#f1d4Af",
    "transition metal"     : "#599d7A",
}

TOOLTIPS = [
    ("Name", "@name"),
    ("Atomic number", "@{atomic number}"),
    ("Atomic mass", "@{atomic mass}"),
    ("Type", "@metal"),
    ("CPK color", "$color[hex, swatch]:CPK"),
    ("Electronic configuration", "@{electronic configuration}"),
]

p = figure(title="Periodic Table (omitting LA and AC Series)", width=1000, height=450,
           x_range=groups, y_range=list(reversed(periods)),
           tools="hover", toolbar_location=None, tooltips=TOOLTIPS)

r = p.rect("group", "period", 0.95, 0.95, source=df, fill_alpha=0.6, legend_field="metal",
           color=factor_cmap('metal', palette=list(cmap.values()), factors=list(cmap.keys())))

text_props = dict(source=df, text_align="left", text_baseline="middle")

x = dodge("group", -0.4, range=p.x_range)

p.text(x=x, y="period", text="symbol", text_font_style="bold", **text_props)

p.text(x=x, y=dodge("period", 0.3, range=p.y_range), text="atomic number",
       text_font_size="11px", **text_props)

p.text(x=x, y=dodge("period", -0.35, range=p.y_range), text="name",
       text_font_size="7px", **text_props)

p.text(x=x, y=dodge("period", -0.2, range=p.y_range), text="atomic mass",
       text_font_size="7px", **text_props)

p.text(x=["3", "3"], y=["VI", "VII"], text=["LA", "AC"], text_align="center", text_baseline="middle")

p.outline_line_color = None
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_standoff = 0
p.legend.orientation = "horizontal"
p.legend.location ="top_center"
p.hover.renderers = [r] # only hover element boxes

show(p)

## Altair

In [4]:
import altair as alt

In [5]:
keep_columns = ['group', 'electronegativity', 'atomic number', 'atomic mass', 'symbol']
elements_ = elements.loc[elements.group != '-', keep_columns].copy()
elements_['group'] = elements_['group'].astype(int)

In [6]:
slider = alt.binding_range(min=1, max=18, step=1, name='group:')
selector = alt.selection_single(fields=['group'], bind=slider, init={'group': 17})

chart = (
    alt.Chart(elements_, title=f"Periodic electronegativity trends within groups").mark_point()
    .encode(
        x=alt.X('symbol:N', sort=elements.symbol.to_numpy()),
        y=alt.Y('electronegativity:Q', scale=alt.Scale(domain=(0.5, 4.5))),
        tooltip=['symbol','atomic number','atomic mass']
    )
    .add_selection(selector)
    .transform_filter(selector)
    .properties(width=400, height=200)
    .interactive()
)
chart

alt.Chart(...)